# Introducción

El propósito de este Notebook es:

- Visualizar la localización de los puntos de concentración de accidentes mortales, que han tenido fallecidos, con el fin de detectar si hay tramos donde se concentren accidentes en las carreteras/calles de Madrid y donde se situan.
- Visualizar la localización de los puntos donde se encuentran los radares situados en Madrid.
- Cruzar la localización de ambos tipos de puntos, con el fin de saber si el posicionamiento de los radares se establece de acuerdo con la posibilidad o no de que ocurran accidentes mortales.



# Tratamiento Inicial de los datos

Instalamos primeramente las librerias que nos hacen falta

In [1]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd, os
import numpy as np
import geocoder
import folium

Vamos a utilizar la API de Google para geolocalizar los accidentes.


In [4]:
api_key = 'Escribe tu Api Key'

Nota: Para saber como conseguir tu api key, puedes mirar el documento adjunto en este mismo repositorio, llamado 'Pasos de la Api Key'

Cargamos el DataSet que ya hemos tratado anteriormente

In [5]:
pathin = "3. DF/"
DF = pd.read_csv(os.path.join(pathin, "DF.csv"),sep=",",skip_blank_lines=False,parse_dates= ["Fecha"]).dropna(axis = 0)
DF

,No_Parte,Fecha,Hora,Direccion,Distrito,Tipo_Accidente,Tipo_Vehiculo,Tipo_Persona,Tramo_Edad,Sexo,...,CPFA_Nieve,CPFA_Lluvia,CPFA_Hielo,CPFA_Niebla,CPSV_Mojada,CPSV_Aceite,CPSV_Barro,CPSV_Grava_Suelta,CPSV_Hielo,CPSV_Seca_Y_Limpia
0,2019S000020,2019-01-01,23:30,"CALL. FUENCARRAL, 149, Madrid",Chamberí,Caída ciclomotor,Ciclomotor,Conductor,DE 25 A 29 AÑOS,0,...,0,0,0,0,0,0,0,0,0,0
1,2019S000017,2019-01-01,22:15,"CALL. OCA / CALL. PINZON, , Madrid",Carabanchel,Colisión doble,Ciclomotor,Conductor,DE 35 A 39 AÑOS,0,...,0,0,0,0,0,0,0,0,0,0
2,2019S001812,2019-01-01,21:40,"CALL. BAILEN / CUSTA. SAN VICENTE, , Madrid",Centro,Colisión doble,Turismo,Conductor,DE 30 A 34 AÑOS,1,...,0,0,0,0,0,0,0,0,0,0
3,2019S001336,2019-01-01,20:25,"CALL. MESONERO ROMANOS, 1, Madrid",Centro,Atropello,Turismo,Peatón,DE 55 A 59 AÑOS,0,...,0,0,0,0,0,0,0,0,0,0
4,2019S000028,2019-01-01,19:15,"PASEO RECOLETOS-PLAZA COLON, , Madrid",Salamanca,Colisión doble,Turismo,Conductor,DE 50 A 54 AÑOS,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266424,2019/18,2018-12-31,21:30,"CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA, , Ma...",Salamanca,Colisión doble,Turismo,Pasajero,DE 30 A 34 AÑOS,0,...,0,0,0,0,0,0,0,0,0,1
266425,2019/18,2018-12-31,21:30,"CALLE DE VELAZQUEZ - CALLE DE VILLANUEVA, , Ma...",Salamanca,Colisión doble,Auto-taxi,Conductor,DE 40 A 44 AÑOS,0,...,0,0,0,0,0,0,0,0,0,1
266426,2019/53,2018-12-31,23:30,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,Hortaleza,Otras causas,Turismo,Conductor,DE 21 A 24 AÑOS,1,...,0,0,0,0,0,0,0,0,0,1
266427,2019/53,2018-12-31,23:30,CALLE DE JOAQUIN FERNANDEZ LEIVA - CALLE DE SI...,Hortaleza,Otras causas,Turismo,Conductor,DE 50 A 54 AÑOS,0,...,0,0,0,0,0,0,0,0,0,1


Seleccionamos solo aquellos accidentes que han supuesto uno o mas fallecidos

In [6]:
DF_Fallecido = DF[(DF['Lesividad']==(3))]
DF_Geo_Agrupado = DF_Fallecido.iloc[:,[0,3,4]].drop_duplicates('No_Parte').reset_index(drop=True)
DF_Geo_Agrupado

,No_Parte,Direccion,Distrito
0,2019S004400,"CALL. SINESIO DELGADO, 84, Madrid",Moncloa - Aravaca
1,2019S006720,"CALL. GINZO DE LIMIA, 21, Madrid",Fuencarral - El Pardo
2,2019S007884,CALL. FRANCISCO SILVELA / CALL. LOPEZ DE HOYOS...,Salamanca
3,2019S008072,"M-11 PK 1, , Madrid",Ciudad Lineal
4,2019S012483,"AVDA. DOCTOR FEDERICO RUBIO Y GALI, 55, Madrid",Moncloa - Aravaca
...,...,...,...
294,2018/14427,"AVENIDA DEL PARTENON - GLORIETA DE LUXEMBURGO,...",Barajas
295,2018/15069,"AVENIDA DE LOS ROSALES KM., 5000, Madrid",Villaverde
296,2018/15901,"AUTOVIA M-30 CALZADA 1 KM., 11000, Madrid",Arganzuela
297,2018/15387,"CALLE DE BRAVO MURILLO NUM, 126, Madrid",Tetuán


Hay que tratar las direcciones que contienen info de carreteras, porque sino la obtención de Latitud Longitud no va a ser la correcta. Es por ello, que separaremos en dos momentaneamente el DataSet, para tratar dichos campos, modificaremos el campo direccion y juntaremos ambos datasets en uno, otra vez.

#### Dataset con datos de carretera

In [7]:
DF_Geo_Agrupado_Carr = DF_Geo_Agrupado[DF_Geo_Agrupado['Direccion'].str.contains("KM", case=False)]
DF_Geo_Agrupado_Carr

,No_Parte,Direccion,Distrito
34,2010/732,"AUTOVIA M-30 CALZADA 2 KM., 9100, Madrid",Moratalaz
43,2010/2892,"AVENIDA DE PORTUGAL FL KM., 4400, Madrid",Latina
44,2010/4714,"AUTOVIA M-30 CALZADA 1 KM., 10300, Madrid",Retiro
47,2010/5494,"AUTOVIA M-30 CALZADA 2 KM., 2900, Madrid",Ciudad Lineal
54,2010/7532,"AUTOVIA M-30 CALZADA 2 KM., 32000, Madrid",Fuencarral - El Pardo
57,2010/8533,"AUTOVIA M-30 CALZADA 2 KM., 32200, Madrid",Chamartín
58,2010/8809,"AUTOVIA M-30 CALZADA 1 KM., 6000, Madrid",Salamanca
65,2010/11731,"AUTOVIA M-30 CALZADA 1 KM., 10400, Madrid",Retiro
70,2011/1698,"AVENIDA DE PUERTA DE HIERRO KM., 5800, Madrid",Moncloa - Aravaca
85,2011/7705,"AVENIDA DEL MEDITERRANEO KM., 4900, Madrid",Villa de Vallecas


#### Dataset sin datos de carretera

In [8]:
DF_Geo_Agrupado_NoCarr = DF_Geo_Agrupado[DF_Geo_Agrupado['Direccion'].str.contains("KM", case=False) == False]
DF_Geo_Agrupado_NoCarr

,No_Parte,Direccion,Distrito
0,2019S004400,"CALL. SINESIO DELGADO, 84, Madrid",Moncloa - Aravaca
1,2019S006720,"CALL. GINZO DE LIMIA, 21, Madrid",Fuencarral - El Pardo
2,2019S007884,CALL. FRANCISCO SILVELA / CALL. LOPEZ DE HOYOS...,Salamanca
3,2019S008072,"M-11 PK 1, , Madrid",Ciudad Lineal
4,2019S012483,"AVDA. DOCTOR FEDERICO RUBIO Y GALI, 55, Madrid",Moncloa - Aravaca
...,...,...,...
291,2018/12229,PASEO DEL PRADO - PLAZA DE CANOVAS DEL CASTILL...,Retiro
292,2018/13372,"AVENIDA DE ENTREVIAS NUM, 100, Madrid",Puente de Vallecas
293,2018/14106,"CALLE DE FRANCISCO SILVELA NUM, 27, Madrid",Salamanca
294,2018/14427,"AVENIDA DEL PARTENON - GLORIETA DE LUXEMBURGO,...",Barajas


#### Tratamiento de las direcciones de carreteras

In [9]:
DF_Geo_Agrupado_Carr['Direccion']= DF_Geo_Agrupado_Carr['Direccion'].replace(to_replace=['KM.', 'CALZADA 1','CALZADA 2','AUTOVIA'],value= '', regex=True)

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
df = pd.DataFrame(DF_Geo_Agrupado_Carr.Direccion.str.split(',',3).tolist(),
                                   columns = ['a','b','c'])
df

,a,b,c
0,M-30,9100,Madrid
1,AVENIDA DE PORTUGAL FL,4400,Madrid
2,M-30,10300,Madrid
3,M-30,2900,Madrid
4,M-30,32000,Madrid
5,M-30,32200,Madrid
6,M-30,6000,Madrid
7,M-30,10400,Madrid
8,AVENIDA DE PUERTA DE HIERRO,5800,Madrid
9,AVENIDA DEL MEDITERRANEO,4900,Madrid


In [11]:
df['b'] = round(df['b'].astype(int)/1000,0)
df['b'] = df['b'].astype('str')

In [12]:
df['b']= df['b'].replace(to_replace=['.0', ''],value= '', regex=True)

In [13]:
DF_Geo_Agrupado_Carr['Direccion']  = (df['a'].map(str)+','+df['b'].map(str)+df['c'].map(str)).values
DF_Geo_Agrupado_Carr['Direccion']

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


34                                M-30  ,9 Madrid
43               AVENIDA DE PORTUGAL FL ,4 Madrid
44                                 M-30  , Madrid
47                                M-30  ,3 Madrid
54                               M-30  ,32 Madrid
57                               M-30  ,32 Madrid
58                                M-30  ,6 Madrid
65                                 M-30  , Madrid
70          AVENIDA DE PUERTA DE HIERRO ,6 Madrid
85             AVENIDA DEL MEDITERRANEO ,5 Madrid
90                               M-30  ,25 Madrid
94         CARRETERA DE COLMENAR VIEJO ,14 Madrid
100            AVENIDA DEL MEDITERRANEO ,5 Madrid
111               CARRETERA DE CASTILLA ,0 Madrid
115                              M-30  ,32 Madrid
156    CARRETERA DE VICALVARO A COSLADA ,4 Madrid
164                               M-30  ,8 Madrid
166           AVENIDA DE AMERICA SALIDA ,0 Madrid
169                               M-30  ,0 Madrid
174                              M-30  ,32 Madrid


#### Se juntan otra vez los dos datasets

In [14]:
DF_Geo_Agrupado = pd.merge(DF_Geo_Agrupado_Carr,DF_Geo_Agrupado_NoCarr, how = 'outer')
DF_Geo_Agrupado

,No_Parte,Direccion,Distrito
0,2010/732,"M-30 ,9 Madrid",Moratalaz
1,2010/2892,"AVENIDA DE PORTUGAL FL ,4 Madrid",Latina
2,2010/4714,"M-30 , Madrid",Retiro
3,2010/5494,"M-30 ,3 Madrid",Ciudad Lineal
4,2010/7532,"M-30 ,32 Madrid",Fuencarral - El Pardo
...,...,...,...
294,2018/12229,PASEO DEL PRADO - PLAZA DE CANOVAS DEL CASTILL...,Retiro
295,2018/13372,"AVENIDA DE ENTREVIAS NUM, 100, Madrid",Puente de Vallecas
296,2018/14106,"CALLE DE FRANCISCO SILVELA NUM, 27, Madrid",Salamanca
297,2018/14427,"AVENIDA DEL PARTENON - GLORIETA DE LUXEMBURGO,...",Barajas


# Bucle de Geolocalización

Hacemos un bucle en el que geolocalizamos los accidentes mortales. Dicho bucle utilizando la libreria Geocoder, conectará con la API de Geocoding de Google, devolviendonos longitud y latitud de cada uno de los accidentes mortales.

In [15]:
i = 0
data = pd.DataFrame()
for i in range(0,len(DF_Geo_Agrupado)):
    g =pd.DataFrame([geocoder.google(DF_Geo_Agrupado.iloc[i,1],key=api_key).latlng])
    data = data.append(g)

In [ ]:
DF_Geo_Agrupado['Lat'],DF_Geo_Agrupado['Long']= data.values[:,0],data.values[:,1]

In [17]:
DF_Geolocalizacion_Fallecidos = DF_Geo_Agrupado

# Depurar Geolocalizaciones

Parte del trabajo realizado en este DataSet, ha consistido en ver, que geolocalizaciones (puntos Lat, Long) han quedado mal geolocalizadas por diferentes razones y quitarlas del DataSet. 
Explicamos a continuación que acciones se han tomado, para hacer esta limpieza:

1. Estudio de puntos negros: Se ha comprobado previamente que geolocalizaciones repetidas, no significan que halla por defecto un punto negro especificamente. Se puede dar el caso que cuando coincidan las geolocalizaciones sea que, con la dirección que le hemos dado, no ha encontrado el punto concreto que le hemos dado y da otro por defecto, que lo comparten tambien otras ubicaciones que no consiguen ser geolocalizadas correctamente.
2. Coordenadas fuera de rango: Quitamos geolocalizaciones que quedan fuera del mapa de coordenadas de Madrid
3. Valores NaN: Quitamos geolocalizaciones que tienen valor NAN

#### 1.Estudio de puntos negros

In [18]:
DF_Geolocalizacion_Fallecidos['LatLong'] = (DF_Geolocalizacion_Fallecidos['Lat'].map(str)+'/'+DF_Geolocalizacion_Fallecidos['Long'].map(str)).values
DF = DF_Geolocalizacion_Fallecidos['LatLong'].value_counts()
DF[DF>1]

40.4210557/-3.7224778          11
40.4511748/-3.6648801           5
40.3917301/-3.6748315           5
40.4167754/-3.7037902           3
40.47925/-3.712499999999999     3
40.4681102/-3.7117872           2
40.4323271/-3.6976173           2
40.3530968/-3.6931236           2
40.4076213/-3.6770901           2
40.443753/-3.7312618            2
40.3686816/-3.6939234           2
40.4125848/-3.7344011           2
Name: LatLong, dtype: int64

Se han chequeado los puntos negros y sabemos que tenemos que desechar por mal geolocalizadas, las siguientes filas:

In [19]:
Valores_Invalidos=DF_Geolocalizacion_Fallecidos[DF_Geolocalizacion_Fallecidos.LatLong.isin(['40.421055700000004/-3.7224778', '40.39185/-3.67562','40.4167754/-3.7037902000000003'])]

In [ ]:
Valores_Invalidos_Index = Valores_Invalidos.index
DF_Geolocalizacion_Fallecidos.drop(Valores_Invalidos_Index,inplace=True)
DF_Geolocalizacion_Fallecidos.drop(['LatLong'],inplace=True,axis=1)
DF_Geolocalizacion_Fallecidos.reset_index()

#### 2. Coordenadas fuera de rango

In [21]:
DF_Geolocalizacion_Fallecidos['Lat'].sort_values(ascending=True),DF_Geolocalizacion_Fallecidos['Long'].sort_values(ascending=True)

(55     10.393228
 32     38.314750
 260    40.195412
 96     40.345644
 251    40.345787
          ...    
 236    40.495644
 250    40.498332
 120    40.507030
 11     40.527871
 125          NaN
 Name: Lat, Length: 299, dtype: float64, 55    -75.483231
 129    -3.810640
 51     -3.810017
 65     -3.795387
 178    -3.774605
          ...    
 266    -3.579336
 262    -3.577733
 32     -3.570256
 250    -3.567598
 125          NaN
 Name: Long, Length: 299, dtype: float64)

In [22]:
DF_Geolocalizacion_Fallecidos.drop([55,114,32],inplace=True)

#### 3.Valores NaN: Quitamos geolocalizaciones que tienen valor NAN

In [23]:
DF_Geolocalizacion_Fallecidos.dropna(inplace=True)
DF_Geolocalizacion_Fallecidos= DF_Geolocalizacion_Fallecidos.reset_index(drop=True)

# Visualización con Libreria Folium

Las coordenadas obtenidas, se visualizan en un mapa utilizando la libreria Folium

En un primer momento, se geolocalizo por puntos los accidentes mortales en Madrid, pero solo es util esta visualizacion, si queremos ver todos los accidentes que ha habido en Madrid. Pero lo que nos interesa es saber si ha habido algun tramo donde se concentren los accidentes y asi es dificil...
Es por ello, que vamos a redondear las cifras de longitud y latitud. 
De esta manera, podremos agrupar zonas en areas homogeneas y ver donde se concentran mas los accidentes.

In [ ]:
DF_Geo_truncado = DF_Geolocalizacion_Fallecidos
DF_Geo_truncado['Lat'] = round(DF_Geolocalizacion_Fallecidos['Lat'],2)
DF_Geo_truncado['Long'] = round(DF_Geolocalizacion_Fallecidos['Long'],2)
DF_Geo_truncado.reset_index()

In [25]:
DF_Geo_truncado2 = DF_Geo_truncado.groupby(['Lat','Long']).size().reset_index()
DF_Geo_truncado2['Freq'] = DF_Geo_truncado2[0]
DF_Geo_truncado2.drop(0,axis=1,inplace=True)

In [26]:
DF_Geo_truncado2['Freq'].unique()

array([ 1,  2,  3,  4,  7, 13,  8, 10,  6,  5], dtype=int64)

Visualizamos aqui, los accidentes agrupados por zona.

In [27]:
i = 0
    
e = folium.Map(
    location=[DF_Geo_truncado2['Lat'][5],DF_Geo_truncado2['Long'][5]],
    zoom_start=11.5,
    tiles='Stamen Terrain'

    
)

for i in range(0, len(DF_Geo_truncado2)):
    folium.Circle([DF_Geo_truncado2['Lat'][i], DF_Geo_truncado2['Long'][i]],
                  radius = float(DF_Geo_truncado2['Freq'][i]*50),
                  color='red',
                  fill=True,
                  fill_color='red').add_to(e)
    
e

Nota: En Github no es posible ver directamente el output de Folium, pero para hacernos una idea. Se adjunta captura. No obstante, se recomiendo abrir el mapa descargandose el este cogido

https://github.com/EvaSanchezPalomo/TFM/blob/master/4.%20Imagenes%20Readme/7.PNG

Consideraremos punto negro, aquel que tenga 5 o mas accidentes mortales.De manera, que vemos que hay 8 zonas donde se concentra accidentes mortales

In [28]:
DF_Geo_PuntosNegros = DF_Geo_truncado2[DF_Geo_truncado2['Freq'] >= 5].reset_index()
DF_Geo_PuntosNegros

,index,Lat,Long,Freq
0,27,40.39,-3.67,7
1,47,40.41,-3.73,7
2,59,40.42,-3.72,13
3,61,40.42,-3.70,8
4,74,40.43,-3.67,10
5,83,40.44,-3.70,6
6,85,40.44,-3.68,7
7,96,40.45,-3.70,6
8,100,40.45,-3.66,5
9,120,40.48,-3.71,6


In [29]:
i = 0
    
e = folium.Map(
    location=[DF_Geo_PuntosNegros['Lat'][5],DF_Geo_PuntosNegros['Long'][5]],
    zoom_start=11.5,
    tiles='Stamen Terrain'

    
)

for i in range(0, len(DF_Geo_PuntosNegros)):
    folium.Circle([DF_Geo_PuntosNegros['Lat'][i], DF_Geo_PuntosNegros['Long'][i]],
                  radius = float(DF_Geo_PuntosNegros['Freq'][i]*50),
                  color='red',
                  fill=True,
                  fill_color='red').add_to(e)
    
e

![](https://github.com/EvaSanchezPalomo/TFM/blob/master/4.%20Imagenes%20Readme/4.PNG)

Destaca una zona de Madrid por su concentración de accidentes mortales y es el barrio de Guindalera

In [30]:
DF_Geo_truncado[(DF_Geo_truncado['Lat']==40.43) & (DF_Geo_truncado['Long']==-3.67)]

,No_Parte,Direccion,Distrito,Lat,Long
65,2019S036952,INS. AVENIDA DE LOS TOREROS / CALL. RAFAELA BO...,Salamanca,40.43,-3.67
141,2012/9259,CALLE DE FRANCISCO SILVELA - PLAZA DE MANUEL B...,Salamanca,40.43,-3.67
155,2013/2839,CALLE DE FRANCISCO SILVELA - CALLE DE DIEGO DE...,Salamanca,40.43,-3.67
166,2013/7459,CALLE DE CARTAGENA - CALLE DE MARTINEZ IZQUIER...,Salamanca,40.43,-3.67
188,2014/8249,"CALLE DE FRANCISCO SILVELA NUM, 11, Madrid",Salamanca,40.43,-3.67
191,2014/9430,"CALLE DE ARDEMANS NUM, 18, Madrid",Salamanca,40.43,-3.67
205,2015/6884,"CALLE DE ALCALA - CALLE DEL CARDENAL BELLUGA, ...",Salamanca,40.43,-3.67
266,2018/725,"CALLE DE ALCALA NUM, 211, Madrid",Salamanca,40.43,-3.67
284,2018/7775,"CALLE DE ALCALA NUM, 174, Madrid",Salamanca,40.43,-3.67
292,2018/14106,"CALLE DE FRANCISCO SILVELA NUM, 27, Madrid",Salamanca,40.43,-3.67


La otra ubicacion esta situada en el barrio de Puerta del Angel. 

In [31]:
DF_Geo_truncado[(DF_Geo_truncado['Lat']==40.41) & (DF_Geo_truncado['Long']==-3.73)]

,No_Parte,Direccion,Distrito,Lat,Long
1,2010/2892,"AVENIDA DE PORTUGAL FL ,4 Madrid",Latina,40.41,-3.73
20,2015/3201,"PASEO DE EXTREMADURA ,6 Madrid",Latina,40.41,-3.73
22,2015/6015,"AVENIDA DE PORTUGAL FT ,5 Madrid",Latina,40.41,-3.73
86,2010/7456,"PASEO DE EXTREMADURA - CALLE DE ANTILLON, , Ma...",Latina,40.41,-3.73
207,2015/7577,"CALLE DE GUADARRAMA - PASEO DE EXTREMADURA, , ...",Latina,40.41,-3.73
257,2017/9695,"PASEO DE EXTREMADURA NUM, 2, Madrid",Latina,40.41,-3.73
276,2018/5809,"PASEO DE EXTREMADURA NUM, 7, Madrid",Latina,40.41,-3.73


# Geolocalización de Radares

Se ha buscado dentro de la web de datos abiertos del ayuntamiento una Base de Datos de Radares.
Dicha base no incluye la información de cuenta se instaló el radar, cosa que hubiera sido interesante. Esto hubiera permitido saber si el ayuntamiento ante la situación de encontrarse un punto peligroso hubiera tomado medidas para ello.
Se ha intentado buscar en otras fuentes, pero no se ha encontrado.

In [39]:
pathin = "1. Raw/"
Radares = pd.read_excel(os.path.join(pathin, "RADARES FIJOS_vDTT.xlsx"),sep=";",encoding='Latin1')
Radares

,Nº\nRADAR,Ubicacion,Carretara o vial,UBICACIÓN\nCalle 30,PK,Sentido,Tipo,X (WGS84),Y (WGS84),Longitud,Latitud,Coordenadas
0,1,"M-30, ENTRADA DE LA A-1, SENTIDO PUENTE DE VEN...",M-30,00NC50,0+500,Calzada interior,Fijo carriles 3 y 4,442843.000000,4.481180e+06,-3.674338,40.479341,DGP
1,2,"M-30, ENTRADA DE LA A-1, SENTIDO PUENTE DE VEN...",M-30,00NC50,0+500,Calzada interior,Fijo carriles 3 y 4,442847.000000,4.481180e+06,-3.674291,40.479342,DGP
2,3,"PASO INFERIOR COSTA RICA, SENTIDO PRÍNCIPE DE ...",NaN,NaN,NaN,NaN,NaN,443295.000000,4.478910e+06,-3.668803,40.458923,DGP
3,4,"M-30, PTE. DE VENTAS. SENTIDO SUR A O´DONNELL ...",M-30,M-30,6+700,Tronco calzada interior,"Fijo carriles 1, 2 y 3",444002.000000,4.475270e+06,-3.660144,40.426180,DGP
4,5,"M-30, PTE. DE VENTAS. SENTIDO SUR A O´DONNELL ...",M-30,M-30,6+700,Tronco calzada interior,"Fijo carriles 1, 2 y 3",444009.000000,4.475271e+06,-3.660061,40.426190,DGP
5,6,"M-30, ENLACE O´DONNELL SENTIDO NORTE A PTE. DE...",M-30,M-30,7+800,Tronco calzada exterior,Fijo carriles 2 y 3,444069.000000,4.474210e+06,-3.659261,40.416636,DGP
6,7,"M-30, ENLACE O´DONNELL SENTIDO NORTE A PTE. DE...",M-30,M-30,7+800,Tronco calzada exterior,Fijo carriles 2 y 3,444066.000000,4.474211e+06,-3.659296,40.416644,DGP
7,8,"TÚNEL BAIPÁS, SENTIDO NORTE A PTE. DE VENTAS (...",M-30,10XC30,10+300,Calzada exterior,Fijo carril 1,NaN,NaN,-3.670048,40.397352,Calle30
8,9,"TÚNEL BAIPÁS, SENTIDO NORTE A PTE. DE VENTAS (...",M-30,10XC30,10+300,Calzada exterior,Fijo carril 3,NaN,NaN,-3.669973,40.397323,Calle30
9,10,"ENTRADA TÚNEL M-30, DESDE NUDO SUR HACIA PTE. ...",M-30,13NL25,13+250,Calzada interior,Fijo carril 3,NaN,NaN,-3.694998,40.386787,Calle30


Se muestra a continuación, la geolocalización exacta de los radares.

In [41]:
i = 0
    
e = folium.Map(
    location=[DF_Geo_truncado2['Lat'][1],DF_Geo_truncado2['Long'][1]],
    zoom_start=11.5,
    tiles='Stamen Terrain'

    
)


    


for i in range(0,len(Radares)):
    folium.Marker([Radares['Latitud'][i], Radares['Longitud'][i]]).add_to(e)
    
e

![](https://github.com/EvaSanchezPalomo/TFM/blob/master/4.%20Imagenes%20Readme/5.PNG)

Para poder cruzar su localización con la de los puntos donde se concentran accidentes, en necesario hacer el mismo ejercicio de redondeo de las latitudes y las longitudes.

In [47]:
Radares['Latitud'] = round(Radares['Latitud'],2)
Radares['Longitud'] = round(Radares['Longitud'],2)

In [48]:
i = 0
    
e = folium.Map(
    location=[DF_Geo_PuntosNegros['Lat'][5],DF_Geo_PuntosNegros['Long'][5]],
    zoom_start=11.5,
    tiles='Stamen Terrain'

    
)

for i in range(0,len(DF_Geo_PuntosNegros)):
    folium.Circle([DF_Geo_PuntosNegros['Lat'][i], DF_Geo_PuntosNegros['Long'][i]],
                  radius = float(DF_Geo_PuntosNegros['Freq'][i]*50),
                  color='red',
                  fill=True,
                  fill_color='red').add_to(e)

    


for i in range(0,len(Radares)):
    folium.Marker([Radares['Latitud'][i], Radares['Longitud'][i]]).add_to(e)
    
e

![](https://github.com/EvaSanchezPalomo/TFM/blob/master/4.%20Imagenes%20Readme/6.PNG)

Conclusión: de los puntos de concentración de accidentes mortales (más de 5 accidentes mortales), solo uno se encuentra cerca de los radares. Por lo que no podemos determinar que la decisión a la hora de establecer un radar, sea la geolocalización de los accidentes mortales.